# Double integrating sphere measurements

**Scott Prahl**

**June 2024**

In [1]:
import numpy as np

import iadpython.sphere

%config InlineBackend.figure_format='retina'

## Double Integrating Spheres

A cross-section of the double integrating sphere setup used to measure reflection and transmission simultaneously is

<center>
    <img src="two-sphere-basic.png" width="80%" />
</center>

The spheres may be different sizes with different wall reflectivities, but the port size for the sample for both spheres must be the same.

In addition to the usual redistribution effects of each sphere, the sample allows the two spheres to interact with one another as light passes through the sample.  This notebook derives the equations that describe this effect.

### Reflection sphere

The light falling on the detector port in the reflection sphere is
$$
P^r = a_d^r \sum_{k=0}^{\infty} P_{2k}
$$
where $P_k$ is the diffuse light power in the sphere after $k$ passes through the sample. Even numbers of passes describe light in the reflectance sphere and odd numbers correspond to light in the transmission sphere.

The diffuse light $P_0$ describes the light in the reflection sphere that has never passed fully through the sample any times.  It should be identical to the single sphere value.

The diffuse light $P_1$ describes the light in the transmission sphere that has passed  through the sample once.  This will be combination of the light passing through the sample due to collimated light incidence on the sample and diffuse light incident on the sample (arising from multiple bounces within the reflection sphere).  Both $P_0$ and $P_1$ will be described in greater detail below.

The power in the reflectance sphere for light that has traversed the sample twice is
$$
P_2 = G^r \cdot \mathrm{UTU} \cdot a_s \cdot P_1
$$
this can be generalized to
$$
P_{2k} =  G^r \cdot \mathrm{UTU} \cdot a_s\cdot P_{2k-1}
\qquad\qquad k\ge1
$$

### Transmission Sphere

The light in the transmission sphere for light thas has made $2k-1$ passes through the sample is
$$
P_{2k-1} =  G^t \cdot \mathrm{UTU} \cdot a_s\cdot P_{2k-2}
\qquad\qquad k>1
$$
The power in the reflectance sphere
$$
P_{2k} =  a_s^2 G^r G^t \mathrm{UTU}^2 \cdot P_{2k-2}
\qquad\qquad k>1
$$
or more generally
$$
P_{2k} =  (a_s^2 G^r G^t  \mathrm{UTU}^2)^{k-1} \cdot P_2
\qquad\qquad k>0
$$

If $\alpha=G^r G^t a_s^2 \mathrm{UTU}^2$, then
$$
P^r = P_0 + P_2 + P_4 + \cdots = P_0 + P_2\left[1 + \alpha + \alpha^2 +\cdots \right]
$$
and summing we get
$$
P^r = P_0 + \frac{P_2}{1-\alpha}
$$

To see how the UR1 and UT1 affect this value, we expand and do some algebra
$$
\frac{P^r}{P_i} =  G^r \cdot \mathrm{UR1} + G^r \cdot \mathrm{UTU} \cdot a_s \frac{G^t\mathrm{UT1} + a_s G^r G^t\cdot \mathrm{UR1} \cdot\mathrm{UTU}}{1-\alpha}
$$
or
$$
\frac{P^r}{P_i} =  G^r\cdot\mathrm{UR1}\left[ 1  + \frac{a_s^2  G^r G^t \cdot\mathrm{UTU}^2}{1-\alpha}\right]
+G^r\cdot \mathrm{UT1} \left[\frac{a_s G^t \cdot\mathrm{UTU}}{1-\alpha}\right]
$$
or
$$
\frac{P^r}{P_i} = \mathrm{UR1} \cdot G^r\left[ \frac{1}{1-\alpha}\right]
+ \mathrm{UT1} \cdot G^r\left[\frac{a_s  G^t \cdot\mathrm{UTU}}{1-\alpha} \right]
$$
or
$$
\frac{P^r}{P_i} = \left[ \frac{G^r}{1-\alpha} \right]
\bigg( \mathrm{UR1}  +  \mathrm{UT1} \cdot a_s  G^t \,\mathrm{UTU} \bigg)
$$

The power in the transmission sphere for light that has made a single pass through the sample will also be  will be a combination of the transmitted light for direct illumination and the diffuse light from the first bounce
$$
P_1 = G^t \left[ \mathrm{UT1} \cdot P_i + \mathrm{UTU} \cdot a_s P_0 \right] = \left[ G^t \mathrm{UT1} + a_s G^r G^t \cdot \mathrm{UR1} \cdot\mathrm{UTU} \right]  P_i
$$


## Accounting for other sphere effects

There are three experimental factors affecting the calculations.  First, a fraction of the incident light may hit the reflectance sphere wall first.  Second, not all of the unscattered reflected light might be collected.  Third, the unscattered transmitted light might leave the transmission sphere.

This affects the values of $P_0$ and $P_1$ 

In [152]:
N = 10
P = np.zeros(N)
Psum = np.zeros(N)
Podd = np.zeros(N)

ur1 = 0.3
uru = 0.5
ut1 = 0.5
utu = 0.2

r_sphere = iadpython.sphere.Sphere(100, 40, r_wall=0.98, d_third=10)
t_sphere = iadpython.sphere.Sphere(100, 40, r_wall=0.97, refl=False)

Gr = r_sphere.gain(sample_uru=uru)
Gt = t_sphere.gain(sample_uru=uru)
a_sample = r_sphere.sample.a
alpha = a_sample**2 * Gr * Gt * utu**2

print("a_s     = %7.3f" % (a_sample))
print("G_r     = %7.3f" % Gr)
print("G_t     = %7.3f" % Gt)
print("G_r*a_s = %7.3f" % (Gr * a_sample))
print("⍺       = %10.7f" % alpha)
print("1/(1-⍺) = %10.7f" % (1 / (1 - alpha)))
print()

P[0] = Gr * ur1
P[1] = Gt * (ut1 + utu * a_sample * P[0])

for k in range(1, N // 2):
    P[2 * k] = P[2 * k - 1] * Gr * a_sample * utu
    P[2 * k + 1] = P[2 * k] * Gt * a_sample * utu

Psum[0] = P[0]
for k in range(1, N // 2):
    Psum[2 * k] = P[2 * k] + Psum[2 * k - 2]

Podd[1] = P[1]
for k in range(1, N // 2):
    Podd[2 * k + 1] = P[2 * k + 1] + Podd[2 * k - 1]

Pr = Gr / (1 - alpha) * (ur1 + ut1 * Gt * a_sample * utu)
Pr = P[0] + P[2] / (1 - alpha)
Pt = P[1] + Gt * a_sample * utu * P[2] / (1 - alpha)
Pt = P[1] + alpha * P[1] / (1 - alpha)
Pt = P[1] / (1 - alpha)

print("    k    P[k]       P_even  P_r        P_odd   P_t")
for k in range(10):
    print("%5d %7.4f  -- %7.4f %7.4f -- %7.4f %7.4f" % (k, P[k], Psum[k], Pr, Podd[k], Pt))

a_s     =   0.040
G_r     =  24.010
G_t     =  20.492
G_r*a_s =   0.960
⍺       =  0.0314880
1/(1-⍺) =  1.0325117

    k    P[k]       P_even  P_r        P_odd   P_t
    0  7.2029  --  7.2029  9.4690 --  0.0000 11.7982
    1 11.4267  --  0.0000  9.4690 -- 11.4267 11.7982
    2  2.1948  --  9.3977  9.4690 --  0.0000 11.7982
    3  0.3598  --  0.0000  9.4690 -- 11.7865 11.7982
    4  0.0691  --  9.4668  9.4690 --  0.0000 11.7982
    5  0.0113  --  0.0000  9.4690 -- 11.7978 11.7982
    6  0.0022  --  9.4690  9.4690 --  0.0000 11.7982
    7  0.0004  --  0.0000  9.4690 -- 11.7982 11.7982
    8  0.0001  --  9.4690  9.4690 --  0.0000 11.7982
    9  0.0000  --  0.0000  9.4690 -- 11.7982 11.7982


## Transmission sphere


The light in the transmission sphere that has made $2k+1$ passes through the sample is
$$
P_{2k+1} =  G^t \cdot \mathrm{UTU} \cdot a_s\cdot P_{2k}
\qquad\qquad k\ge1
$$
Since 
$$
P_{2k} =  \alpha^{k-1} \cdot P_2
\qquad\qquad k>0
$$
We can express the contribution for $2k+1$ passes in terms of the first pass
$$
P_{2k+1} = (a_s\cdot  G^t \cdot \mathrm{UTU}) \cdot  \alpha^{k-1} \cdot P_2 = (a_s\cdot  G^t \cdot \mathrm{UTU} )\cdot  \alpha^{k-1} \cdot (G^r \cdot \mathrm{UTU} \cdot a_s \cdot P_1)
\qquad\qquad k>0
$$
which can be simplified down to
$$
P_{2k+1} = (a_s^2\cdot  G^r G^t \cdot \mathrm{UTU}^2 ) \cdot \alpha^{k-1} \cdot P_1 = \alpha^k  P_1
\qquad\qquad k>0
$$

The total light in the transmission sphere is the sum of all the odd terms
$$
P^t = P_1 + P_3 + P_5 + \cdots = P_1 + P_1 \alpha \left[1 + \alpha + \alpha^2 +\cdots \right] = P_1 + P_1 \frac{\alpha}{1-\alpha}
$$
which can be simplified to
$$
P^t  = \frac{P_1}{1-\alpha}
$$



The inital power can be split into two parts:  the light that hits the wall first and the light that hits the sample first.  This gives
$$
P_0 = P_\mathrm{wall} + P_\mathrm{sample} = f_w P_i + (1-f_w) P_i
$$
where $f_w$ is the fraction of light hitting the sphere wall before the sample.  Typically, $f_w=1$ in a well-aligned system.  

The light hitting the wall first will become diffuse light within the sphere, so
$$
P_\mathrm{diffuse, wall} = r_w f_w P_i r_\mathrm{first}
$$
where $r_\mathrm{first}$ accounts for the presence of a baffle for the first bounce.  If there is no baffle then $r_\mathrm{first}=1$ otherwise $r_\mathrm{first}=r_w(1-a_\mathrm{entrance})$.

Of the light that hits the sample, it has two parts, the backscattered and the unscattered (specular) parts.  The backscattered part will be
$$
P_\mathrm{diffuse, backscattered} = (\mathrm{UR1}-R_u) P_\mathrm{sample} = (1-f_w) (\mathrm{UR1}-R_u)  P_i
$$
Finally, only a fraction of the unscattered reflected light light $f_u$ may be collected by the sphere.  This amount is
$$
P_\mathrm{diffuse, unscattered} = f_u R_u r_w P_\mathrm{sample} = (1-f_w) f_u R_u r_w P_i
$$
Putting it all together, we get the diffuse light in the reflectance sphere that has not passed all the way through the sample any times.
$$
P_\mathrm{diffuse} = P_\mathrm{diffuse, wall}+P_\mathrm{diffuse, backscattered}+P_\mathrm{diffuse, unscattered}
$$
$$
P_\mathrm{diffuse} = \big[ r_w f_w  + (1-f_w) (\mathrm{UR1}-R_u)   + (1-f_w) f_u R_u r_w  \big] \cdot P_i
$$

fraction of unscattered reflectance affects how much diffuse light is generated.  Thus
$$
P_\mathrm{diffuse} = (1-f_\mathrm{wall})\left[(\mathrm{UR1} - R_u) + f_u R_u r_w \right] P_i
$$
or
$$
P_\mathrm{diffuse} = (1-f_\mathrm{wall}) (\mathrm{UR1} - R_u) P_i + f_u (1-f_\mathrm{wall}) R_u r_w P_i
$$

The initial diffuse power in the reflectance sphere for an incident power $P_i$ will be
$$
P_\mathrm{diffuse} = (1-f_w) (\mathrm{UR1}- f_u R_u) \cdot P_i + f_w r_w  \cdot P_i +  f_u R_u
$$
and
$$
P_0 = G^r P_i
$$
$$
where $G_r$ is the gain due to the reflectance sphere.
